In [4]:
# 라방바 금주 편성 크롤링

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from selenium.webdriver.common.keys import Keys
import time
from datetime import datetime
import pandas as pd

# 현재 날짜와 시간
now = datetime.now()

today = datetime.today().strftime('%Y-%m-%d')
yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

# 연, 월, 일 따로 추출
year = now.year
month = now.strftime("%m")

url = 'https://live.ecomm-data.com/schedule/hs'
browser = webdriver.Chrome()
browser.get(url)

# 1. 날짜별로 조회.
search_date_list = browser.find_elements('css selector', "ul > li > div.schedule_cell__0ZCso")

for i in range(len(search_date_list)):
    search_date_list = browser.find_elements('css selector', "ul > li > div.schedule_cell__0ZCso")
    search_button = search_date_list[i]
    
    search_button.click()
    time.sleep(2)

    for j in range(5):
        body = browser.find_element('css selector', 'body')
        body.send_keys(Keys.HOME)
        time.sleep(1)
    
    # 크롤링
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    day = search_button.text.split()[0]

    search_date = f'{year}-{month}-{day}'
    print(search_date)


    results = []

    # 날짜별 전체 편성 가져오기
    day_div_list = soup.select('div.schedule_container__Bp3qt') 
    
    # day_div_list[1].select('div.Table_container__cUG9N')[0].select('table.Table_table___jpMW')[0].select('tr')[2].select('td')[1].select('span')[0].text
    
    for day in day_div_list[1:]:
        time_div_list = day.select('div.Table_container__cUG9N')[0].select('table.Table_table___jpMW')[0].select('tr') 
    
        for broad in time_div_list:
            td_list = broad.select('td')
            
            if len(td_list) >= 4 and td_list[1].select('span'):
                try:
                    date_ymd = td_list[1].select('span')[0].text
                    time_hm = td_list[1].select('span')[1].text
                    broad_info = td_list[2].text.strip()
                    company_info = td_list[2].select('span.TableSchedule_adWrap__V_CMK')[0].text.strip() if td_list[2].select('span.TableSchedule_adWrap__V_CMK') else ''
                    category = td_list[3].text.strip()
    
                    # print(date_ymd, time_hm, broad_info, company_info, category)
                    
                    data = [date_ymd, time_hm, broad_info, company_info, category]
                    results.append(data)
    
                except IndexError as e:
                    print("IndexError 발생한 row:", broad)
                    print("에러 메시지:", e)

    # csv로 데이터 저장        
    df = pd.DataFrame(results)
    df.columns = ['date', 'time', 'broad_info', 'company_info', 'category']
    df.to_csv(f'./broad_info/broad_{search_date}.csv', index=False)


2025-04-13
2025-04-14
2025-04-15
2025-04-16
2025-04-17
2025-04-18
2025-04-19
2025-04-20
2025-04-21
